In [1]:
import numpy as np

### Basic definitions on pools and users

In [2]:
pool_size = 2
n = pool_size

In [3]:
class Pool:
    def __init__(self,**kwargs):
        defaults = {"x": np.array([1.]*n), 
                    "l": 1.}
        defaults.update(kwargs)
        self.update(**defaults)
    def update(self, **kwargs):
        if "x" in kwargs:
            self.x = kwargs["x"]
        if "l" in kwargs:
            self.l = kwargs["l"]
    
    def invariant(self):
        result = 1.
        for i in range(n):
            result = result * self.x[i]/self.l
        return result
    
    def balance(self, to_prices=None):
        # reset prices to be given by to_prices 
        if to_prices is None:
            to_prices = np.array([1.]*n)
            
        assert all(to_prices > 0)
        factor = 1
        for i in range(n):
            factor = factor * self.x[i]/to_prices[i]
        self.x = factor**(float(1/n)) * to_prices
    
    def __str__(self):
        return str(self.show())
    
    def show(self):
        return {"x": self.x, "l": self.l}
    

In [4]:
pool_test = Pool()

In [5]:
class User:
    def __init__(self, **kwargs):
        defaults = {"u": np.array([1.]*n), 
                    "k": np.array([0.]*n)}
        defaults.update(kwargs)
        self.update(**defaults)
    def update(self, **kwargs):
        if "u" in kwargs:
            self.u = kwargs["u"]
        if "k" in kwargs:
            self.k = kwargs["k"]

    def __str__(self):
        return str(self.show())
    
    def show(self):
        return {"u": self.u, "k": self.k}

In [6]:
user_test = User()

### Adding and removing liquidity

In [7]:
delta_test = np.array([0.1]*n)

In [8]:
def liquidity_update(delta, x):
    factor = 1.
    for i in range(n):
        factor = factor * (1+delta[i]/x[i])
    exponent = float(1/n)
    return lambda l: l * factor**exponent 

In [9]:
def liquidity_minted(update):
    return lambda l: update(l)-l

In [10]:
# it's not clear that this definitions makes sense in general, 
# in a first draft it can be used for balanced and single token adding

def add_liquidity(delta):
    
    def f(user, pool):
        
        u,k = (user.u, user.k)
        x,l = (pool.x, pool.l)
        
        if sum(delta)==0:
            return (u, k, x, l)
        
        assert all(x>0)
        assert l>0
        
        l_update = liquidity_update(delta, x)
        l_minted = liquidity_minted(liquidity_update)
        
        new_u = u - delta
        assert all(new_u >= 0)
        new_x = x + delta
        new_l = l_update(l)
        minted_l = liquidity_minted(l_update)(l)
        transform_delta = delta/x
        total = sum(transform_delta)
        factor = minted_l/total
        new_k = k + factor * transform_delta
        
        pool.update(x=new_x, l=new_l) 
        user.update(u=new_u, k=new_k)       
    
    return f

In [11]:
add_liquidity_test = add_liquidity(delta_test)

In [12]:
add_liquidity_test(user_test, pool_test)

In [13]:
print(user_test, pool_test)

{'u': array([ 0.9,  0.9]), 'k': array([ 0.05,  0.05])} {'x': array([ 1.1,  1.1]), 'l': 1.1000000000000001}


In [14]:
def remove_liquidity(delta_l, in_type = "balanced", cut_off=0.00000001):
    # in_type supported "balanced" or index of token 0, ..., n-1   
    if in_type!="balanced":
        assert in_type in list(range(n)), "in_type must be balanced or an index 0, ..., pool_size-1"        
    
    def f(user, pool, delta_l=delta_l):
        u, k = user.u, user.k
        x, l = pool.x, pool.l
        
        assert all(x>0)
        #assert l-delta_l>0
        
        if in_type == "balanced":
            
            # computing max possible to remove
            part_delta_l = min([delta_l/n]+list(k))
            delta_l = part_delta_l * n
            new_k = k - np.array([part_delta_l]*n)
            
            new_l = l - delta_l
            update_l_factor = 1 - delta_l/l
            delta = (1 - update_l_factor) * x
            
        else:
            
            # computing max possible remove 
            delta_l = min(delta_l, k[in_type])
            new_l = l - delta_l
            update_l_factor = 1 - delta_l/l
            
            delta = (1 - update_l_factor**n) * x
            for j in range(n):
                if j!=in_type:
                    delta[j]=0
            new_k = k
            for j in range(n):
                if j==in_type:
                    new_k[j] = new_k[j] - delta_l
        
        new_u = u + delta
        new_x = x - delta
        assert all(new_x > 0)
        if not all(new_k >= 0): 
            print("WARNING: {} not all coordinates >= 0".format(new_k))
        assert all(new_k >= -cut_off), "{} not all coordinates >= {}".format(new_k, cut_off)
        
        pool.update(x=new_x, l=new_l)
        user.update(u=new_u, k=new_k)
        
    return f

In [15]:
remove_liquidity(0.05, in_type=0)(user_test, pool_test)

In [16]:
remove_liquidity(0.05, in_type=1)(user_test, pool_test)

In [17]:
# zero fee swaps happen this way
print(user_test, pool_test)

{'u': array([ 0.99772727,  1.00226757]), 'k': array([  4.16333634e-17,   4.16333634e-17])} {'x': array([ 1.00227273,  0.99773243]), 'l': 1.0}


In [18]:
pool_test.invariant()

1.0000000000000002

### Compositions

In [19]:
def c(*args):
    if len(args)==1:
        args = args[0]
        
    args.reverse()
    def f(user, pool):
        for j, g in enumerate(args):
            g(user, pool)
            print("Step {}:  {}, {}\n".format(j, user, pool))
    return f

In [20]:
def mini_swap(delta): 
    chain = []
    for i in range(n):
        chain.append(remove_liquidity(float(delta)/n, in_type=i))
    chain.append(add_liquidity(np.array([float(delta)]*n)))
    
    return c(chain)

In [21]:
r = [float(i/100) for i in range(100)]

for delta in r:
    user_test, pool_test = User(), Pool()
    mini_swap(delta)(user_test, pool_test)
    print(delta)
    print(user_test, pool_test)
    print("********")

Step 0:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 1:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 2:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

0.0
{'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])} {'x': array([ 1.,  1.]), 'l': 1.0}
********
Step 0:  {'u': array([ 0.99,  0.99]), 'k': array([ 0.005,  0.005])}, {'x': array([ 1.01,  1.01]), 'l': 1.01}

Step 1:  {'u': array([ 0.99      ,  0.99997525]), 'k': array([  5.00000000e-03,   4.33680869e-18])}, {'x': array([ 1.01      ,  1.00002475]), 'l': 1.0050000000000001}

Step 2:  {'u': array([ 1.00002475,  0.99997525]), 'k': array([  4.33680869e-18,   4.33680869e-18])}, {'x': array([ 0.99997525,  1.00002475]), 'l': 1.0000000000000002}

0.01
{'u': array([ 1.00002475,  0.99997525]), 'k': array([  4.33680869e-18,   4.33680869e-18])} {'x': array([ 0.99997525,  1.00002475]), 'l': 1.0000000000000002}
********
S

********
Step 0:  {'u': array([ 0.84,  0.84]), 'k': array([ 0.08,  0.08])}, {'x': array([ 1.16,  1.16]), 'l': 1.1599999999999999}

Step 1:  {'u': array([ 0.84      ,  0.99448276]), 'k': array([ 0.08,  0.  ])}, {'x': array([ 1.16      ,  1.00551724]), 'l': 1.0800000000000001}

Step 2:  {'u': array([ 1.00548697,  0.99448276]), 'k': array([ 0.,  0.])}, {'x': array([ 0.99451303,  1.00551724]), 'l': 1.0}

0.16
{'u': array([ 1.00548697,  0.99448276]), 'k': array([ 0.,  0.])} {'x': array([ 0.99451303,  1.00551724]), 'l': 1.0}
********
Step 0:  {'u': array([ 0.83,  0.83]), 'k': array([ 0.085,  0.085])}, {'x': array([ 1.17,  1.17]), 'l': 1.1699999999999999}

Step 1:  {'u': array([ 0.83      ,  0.99382479]), 'k': array([ 0.085,  0.   ])}, {'x': array([ 1.17      ,  1.00617521]), 'l': 1.085}

Step 2:  {'u': array([ 1.00613731,  0.99382479]), 'k': array([ 0.,  0.])}, {'x': array([ 0.99386269,  1.00617521]), 'l': 1.0}

0.17
{'u': array([ 1.00613731,  0.99382479]), 'k': array([ 0.,  0.])} {'x': arra

##### Balancing out between removing liquidity

In [22]:
def balance(user, pool):
    pool.balance()

In [23]:
def mini_swap_balance(delta): 
    chain = []
    for i in range(n):
        chain.append(balance)
        chain.append(remove_liquidity(float(delta)/n, in_type=i))
    chain.append(add_liquidity(np.array([float(delta)]*n)))
    
    return c(chain)

In [24]:
r = [float(i/100) for i in range(20)]

for delta in r:
    user_test, pool_test = User(), Pool()
    mini_swap_balance(delta)(user_test, pool_test)
    print(delta)
    print(user_test, pool_test)
    print("********")

Step 0:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 1:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 2:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 3:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 4:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

0.0
{'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])} {'x': array([ 1.,  1.]), 'l': 1.0}
********
Step 0:  {'u': array([ 0.99,  0.99]), 'k': array([ 0.005,  0.005])}, {'x': array([ 1.01,  1.01]), 'l': 1.01}

Step 1:  {'u': array([ 0.99      ,  0.99997525]), 'k': array([  5.00000000e-03,   4.33680869e-18])}, {'x': array([ 1.01      ,  1.00002475]), 'l': 1.0050000000000001}

Step 2:  {'u': array([ 0.99      ,  0.99997525]), 'k': array([  5.00000000e-03,   4.33680869e-18])}, {'x': array([ 1.005,  1.005]), 'l': 1.00500000000

#### Next miniswap scenario

In [25]:
def mini_swap_2(delta): 
    assert pool_size > 1
    chain = []
    chain.append(remove_liquidity(float(delta), in_type=0))
    chain.append(remove_liquidity(float(delta), in_type=1))
    chain.append(add_liquidity(np.array([float(delta)] + [0]*(n-1))))
    chain.append(add_liquidity(np.array([0, float(delta)] + [0]*(n-2))))
    
    return c(chain)

In [26]:
r = [float(i/100) for i in range(100)]

for delta in r:
    user_test, pool_test = User(), Pool()
    mini_swap_2(delta)(user_test, pool_test)
    print(delta)
    print(user_test, pool_test)
    print("********")

Step 0:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 1:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 2:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 3:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

0.0
{'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])} {'x': array([ 1.,  1.]), 'l': 1.0}
********
Step 0:  {'u': array([ 1.  ,  0.99]), 'k': array([ 0.        ,  0.00498756])}, {'x': array([ 1.  ,  1.01]), 'l': 1.004987562112089}

Step 1:  {'u': array([ 0.99,  0.99]), 'k': array([ 0.00501244,  0.00498756])}, {'x': array([ 1.01,  1.01]), 'l': 1.0099999999999998}

Step 2:  {'u': array([ 0.99      ,  0.99995049]), 'k': array([ 0.00501244,  0.        ])}, {'x': array([ 1.01      ,  1.00004951]), 'l': 1.0050124378879108}

Step 3:  {'u': array([ 1.0000495 ,  0.99995049]), 'k': array([ 0.,  0.])}, {'x': array([ 0.9999505

Step 2:  {'u': array([ 0.87      ,  0.99251515]), 'k': array([ 0.06698542,  0.        ])}, {'x': array([ 1.13      ,  1.00748485]), 'l': 1.0669854187265349}

Step 3:  {'u': array([ 1.00742925,  0.99251515]), 'k': array([ 0.,  0.])}, {'x': array([ 0.99257075,  1.00748485]), 'l': 1.0}

0.13
{'u': array([ 1.00742925,  0.99251515]), 'k': array([ 0.,  0.])} {'x': array([ 0.99257075,  1.00748485]), 'l': 1.0}
********
Step 0:  {'u': array([ 1.  ,  0.86]), 'k': array([ 0.        ,  0.06770783])}, {'x': array([ 1.  ,  1.14]), 'l': 1.0677078252031311}

Step 1:  {'u': array([ 0.86,  0.86]), 'k': array([ 0.07229217,  0.06770783])}, {'x': array([ 1.14,  1.14]), 'l': 1.1399999999999999}

Step 2:  {'u': array([ 0.86      ,  0.99139429]), 'k': array([ 0.07229217,  0.        ])}, {'x': array([ 1.14      ,  1.00860571]), 'l': 1.0722921747968688}

Step 3:  {'u': array([ 1.00853228,  0.99139429]), 'k': array([ 0.,  0.])}, {'x': array([ 0.99146772,  1.00860571]), 'l': 1.0}

0.14
{'u': array([ 1.00853228,  

In [27]:
def mini_swap_balance_2(delta): 
    assert pool_size > 1
    chain = []
    chain.append(remove_liquidity(float(delta), in_type=0))
    chain.append(balance)
    chain.append(remove_liquidity(float(delta), in_type=1))
    chain.append(balance)
    chain.append(add_liquidity(np.array([float(delta)] + [0]*(n-1))))
    chain.append(balance)
    chain.append(add_liquidity(np.array([0, float(delta)] + [0]*(n-2))))
    
    return c(chain)

In [28]:
r = [float(i/100) for i in range(100)]

for delta in r:
    user_test, pool_test = User(), Pool()
    mini_swap_balance_2(delta)(user_test, pool_test)
    print(delta)
    print(user_test, pool_test)
    print("********")

Step 0:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 1:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 2:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 3:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 4:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 5:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

Step 6:  {'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

0.0
{'u': array([ 1.,  1.]), 'k': array([ 0.,  0.])} {'x': array([ 1.,  1.]), 'l': 1.0}
********
Step 0:  {'u': array([ 1.  ,  0.99]), 'k': array([ 0.        ,  0.00498756])}, {'x': array([ 1.  ,  1.01]), 'l': 1.004987562112089}

Step 1:  {'u': array([ 1.  ,  0.99]), 'k': array([ 0.        ,  0.00498756])}, {'x': array([ 1.00498756, 


Step 3:  {'u': array([ 0.92,  0.92]), 'k': array([ 0.03925848,  0.03923048])}, {'x': array([ 1.07848896,  1.07848896]), 'l': 1.0784889608907948}

Step 4:  {'u': array([ 0.92      ,  0.99703394]), 'k': array([ 0.03925848,  0.        ])}, {'x': array([ 1.07848896,  1.00145502]), 'l': 1.0392584763494683}

Step 5:  {'u': array([ 0.92      ,  0.99703394]), 'k': array([ 0.03925848,  0.        ])}, {'x': array([ 1.03925848,  1.03925848]), 'l': 1.0392584763494683}

Step 6:  {'u': array([ 0.99703395,  0.99703394]), 'k': array([ 0.,  0.])}, {'x': array([ 0.96222453,  1.03925848]), 'l': 1.0}

0.08
{'u': array([ 0.99703395,  0.99703394]), 'k': array([ 0.,  0.])} {'x': array([ 0.96222453,  1.03925848]), 'l': 1.0}
********
Step 0:  {'u': array([ 1.  ,  0.91]), 'k': array([ 0.        ,  0.04403065])}, {'x': array([ 1.  ,  1.09]), 'l': 1.0440306508910551}

Step 1:  {'u': array([ 1.  ,  0.91]), 'k': array([ 0.        ,  0.04403065])}, {'x': array([ 1.04403065,  1.04403065]), 'l': 1.0440306508910551}



0.15
{'u': array([ 0.99018532,  0.99018527]), 'k': array([ 0.,  0.])} {'x': array([ 0.93236082,  1.07254614]), 'l': 1.0}
********
Step 0:  {'u': array([ 1.  ,  0.84]), 'k': array([ 0.        ,  0.07703296])}, {'x': array([ 1.  ,  1.16]), 'l': 1.0770329614269007}

Step 1:  {'u': array([ 1.  ,  0.84]), 'k': array([ 0.        ,  0.07703296])}, {'x': array([ 1.07703296,  1.07703296]), 'l': 1.0770329614269007}

Step 2:  {'u': array([ 0.84,  0.84]), 'k': array([ 0.07723099,  0.07703296])}, {'x': array([ 1.23703296,  1.07703296]), 'l': 1.1542639532742518}

Step 3:  {'u': array([ 0.84,  0.84]), 'k': array([ 0.07723099,  0.07703296])}, {'x': array([ 1.15426395,  1.15426395]), 'l': 1.1542639532742518}

Step 4:  {'u': array([ 0.84      ,  0.98892492]), 'k': array([ 0.07723099,  0.        ])}, {'x': array([ 1.15426395,  1.00533904]), 'l': 1.077230991847351}

Step 5:  {'u': array([ 0.84      ,  0.98892492]), 'k': array([ 0.07723099,  0.        ])}, {'x': array([ 1.07723099,  1.07723099]), 'l': 1.07

{'u': array([ 0.97660357,  0.97660299]), 'k': array([ 0.,  0.])} {'x': array([ 0.89754576,  1.11414933]), 'l': 1.0}
********
Step 0:  {'u': array([ 1.  ,  0.75]), 'k': array([ 0.        ,  0.11803399])}, {'x': array([ 1.  ,  1.25]), 'l': 1.1180339887498949}

Step 1:  {'u': array([ 1.  ,  0.75]), 'k': array([ 0.        ,  0.11803399])}, {'x': array([ 1.11803399,  1.11803399]), 'l': 1.1180339887498949}

Step 2:  {'u': array([ 0.75,  0.75]), 'k': array([ 0.118699  ,  0.11803399])}, {'x': array([ 1.36803399,  1.11803399]), 'l': 1.2367329934902982}

Step 3:  {'u': array([ 0.75,  0.75]), 'k': array([ 0.118699  ,  0.11803399])}, {'x': array([ 1.23673299,  1.23673299]), 'l': 1.2367329934902982}

Step 4:  {'u': array([ 0.75     ,  0.9748028]), 'k': array([ 0.118699,  0.      ])}, {'x': array([ 1.23673299,  1.0119302 ]), 'l': 1.1186990047404033}

Step 5:  {'u': array([ 0.75     ,  0.9748028]), 'k': array([ 0.118699,  0.      ])}, {'x': array([ 1.118699,  1.118699]), 'l': 1.1186990047404033}

Ste

### In/rebalance/out losses

In [29]:
def in_rebalance_out(delta): 
    assert pool_size > 1
    chain = []
    chain.append(balance)
    chain.append(remove_liquidity(100*float(delta), in_type=0))
    chain.append(balance)
    chain.append(add_liquidity(np.array([float(delta)] + [0]*(n-1))))
    
    return c(chain)

In [30]:
for scale in [1]:
    delta = scale/100
    user_test, pool_test = User(), Pool()
    in_rebalance_out(delta)(user_test, pool_test)
    print(delta)
    print(user_test, pool_test)
    print((1-user_test.u[0])/delta**2)
    print("*************")

Step 0:  {'u': array([ 0.99,  1.  ]), 'k': array([ 0.00498756,  0.        ])}, {'x': array([ 1.01,  1.  ]), 'l': 1.004987562112089}

Step 1:  {'u': array([ 0.99,  1.  ]), 'k': array([ 0.00498756,  0.        ])}, {'x': array([ 1.00498756,  1.00498756]), 'l': 1.004987562112089}

Step 2:  {'u': array([ 0.99995037,  1.        ]), 'k': array([ 0.,  0.])}, {'x': array([ 0.99503719,  1.00498756]), 'l': 1.0}

Step 3:  {'u': array([ 0.99995037,  1.        ]), 'k': array([ 0.,  0.])}, {'x': array([ 1.,  1.]), 'l': 1.0}

0.01
{'u': array([ 0.99995037,  1.        ]), 'k': array([ 0.,  0.])} {'x': array([ 1.,  1.]), 'l': 1.0}
0.496280979003
*************
